In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import os
from cellpose import io
from liams_funcs import *
from features import *

import pandas as pd

In [ ]:
control_dir = '../control_images/'
penetramax_dir = '../penetramax_images/'

control_fnames = [control_dir + fname for fname in os.listdir(control_dir)]
penetramax_fnames = [penetramax_dir + fname for fname in os.listdir(penetramax_dir)]

In [ ]:
fname1 = '../control_images/001_z26.png'
fname2 = '../penetramax_images/011_z57.png'

In [ ]:
data = np.load('../segments/control/001_z26.png.npy')
image = io.imread('../control_images/001_z26.png')

reds, blues = image[:,:,0], image[:,:,2]
combined_cells, combined_nuclei = data[:,:,0], data[:,:,1]

vals = get_mask_levels(combined_nuclei)
fig, axes = plt.subplots(3, 2, figsize=(12, 12), dpi=300)

for val in vals:
    mask = (combined_nuclei == val)
    red = reds[mask]
    blue = blues[mask]

    counts_red, edges = np.histogram(red, bins=64, range=(0,255), density=True)
    counts_blue, _ = np.histogram(blue, bins=64, range=(0,255), density=True)

    axes[0,0].stairs(counts_red, edges, fill=True, alpha=0.05, color='firebrick')
    axes[0,1].stairs(counts_blue, edges, fill=True, alpha=0.05, color='dodgerblue')

    counts_red, edges = np.histogram(ZtoRGB(gaussian_normalisation(red)), bins=64, range=(0, 255), density=True)
    counts_blue, _= np.histogram(ZtoRGB(gaussian_normalisation(blue)), bins=64, range=(0, 255), density=True)

    axes[1,0].stairs(counts_red, edges, fill=True, alpha=0.05, color='firebrick')
    axes[1,1].stairs(counts_blue, edges, fill=True, alpha=0.05, color='dodgerblue')

    counts_red, edges = np.histogram(ZtoRGB(quantile_normalisation(red)), bins=64, range=(0, 255), density=True)
    counts_blue, _ = np.histogram(ZtoRGB(quantile_normalisation(blue)), bins=64, range=(0, 255), density=True)

    axes[2,0].stairs(counts_red, edges, fill=True, alpha=0.05, color='firebrick')
    axes[2,1].stairs(counts_blue, edges, fill=True, alpha=0.05, color='dodgerblue')

for ax in axes.flatten():
    ax.set_ylim(0)
    ax.set_ylabel('Probability density')
    ax.set_xlabel('Pixel value')
    ax.set_xlim(0, 255)
    
axes[0,0].set_title('No normalisation'); axes[1,0].set_title('Gaussian normalisation'); axes[2,0].set_title('Quantile normalisation')

plt.tight_layout()
plt.savefig('../figures/spectrum_normalisations_control.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
data = np.load('../segments/penetramax/011_z57.png.npy')
image = io.imread('../penetramax_images/011_z57.png')

reds, blues = image[:,:,0], image[:,:,2]
combined_cells, combined_nuclei = data[:,:,0], data[:,:,1]

vals = get_mask_levels(combined_nuclei)
fig, axes = plt.subplots(3, 2, figsize=(12, 12), dpi=300)

for val in vals:
    mask = (combined_nuclei == val)
    red = reds[mask]
    blue = blues[mask]

    counts_red, edges = np.histogram(red, bins=64, range=(0,255), density=True)
    counts_blue, _ = np.histogram(blue, bins=64, range=(0,255), density=True)

    axes[0,0].stairs(counts_red, edges, fill=True, alpha=0.05, color='firebrick')
    axes[0,1].stairs(counts_blue, edges, fill=True, alpha=0.05, color='dodgerblue')

    counts_red, edges = np.histogram(ZtoRGB(gaussian_normalisation(red)), bins=64, range=(0, 255), density=True)
    counts_blue, _= np.histogram(ZtoRGB(gaussian_normalisation(blue)), bins=64, range=(0, 255), density=True)

    axes[1,0].stairs(counts_red, edges, fill=True, alpha=0.05, color='firebrick')
    axes[1,1].stairs(counts_blue, edges, fill=True, alpha=0.05, color='dodgerblue')

    counts_red, edges = np.histogram(ZtoRGB(quantile_normalisation(red)), bins=64, range=(0, 255), density=True)
    counts_blue, _ = np.histogram(ZtoRGB(quantile_normalisation(blue)), bins=64, range=(0, 255), density=True)

    axes[2,0].stairs(counts_red, edges, fill=True, alpha=0.05, color='firebrick')
    axes[2,1].stairs(counts_blue, edges, fill=True, alpha=0.05, color='dodgerblue')

for ax in axes.flatten():
    ax.set_ylim(0)
    ax.set_ylabel('Probability density')
    ax.set_xlabel('Pixel value')
    ax.set_xlim(0, 255)
    
axes[0,0].set_title('No normalisation'); axes[1,0].set_title('Gaussian normalisation'); axes[2,0].set_title('Quantile normalisation')

plt.tight_layout()
plt.savefig('../figures/spectrum_normalisations_drug.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
ds = Dataset(control_fnames, penetramax_fnames)

In [ ]:
ds.performDimReduction()
ds.makeKDE()

In [ ]:
df_unfiltered = ds.makeSelection(q_control=-1, q_drug=-1)
df_05 = ds.makeSelection(q_control=0.05, q_drug=0.05)
df_10 = ds.makeSelection(q_control=0.1, q_drug=0.1)
df_25 = ds.makeSelection(q_control=0.25, q_drug=0.25)

In [ ]:
df_unfiltered.to_csv('feature_data/features_00.csv', index=False)
df_05.to_csv('feature_data/features_05.csv', index=False)
df_10.to_csv('feature_data/features_10.csv', index=False)
df_25.to_csv('feature_data/features_25.csv', index=False)

In [ ]:
X = df_unfiltered.drop('label', axis=1).to_numpy()
X_labels = df_unfiltered.drop('label', axis=1).columns

y = df_unfiltered['label'].to_numpy()

In [ ]:
from sklearn.decomposition import PCA

X_pca = PCA(n_components=2).fit_transform(X)

In [ ]:
plt.figure(dpi=300)
plt.scatter(X_pca[:,0], X_pca[:,1], s=0.5, c=['k' if val==0 else 'r' for val in y])
plt.show()